In [52]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option('mode.chained_assignment', None)

In [55]:
from os import listdir
from os.path import isfile, join
from datetime import datetime
from dateutil.parser import parse

In [56]:
confirmed_global_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
confirmed_states_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'

recovered_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

deaths_global_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
deaths_states_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'

In [57]:
confirmed_global_df = pd.read_csv(confirmed_global_file)
confirmed_states_df = pd.read_csv(confirmed_states_file)
recovered_df = pd.read_csv(recovered_file)
deaths_states_df = pd.read_csv(deaths_states_file)
deaths_global_df = pd.read_csv(deaths_global_file)
confirmed_global_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,110,110,120,170,174,237,273,281,299,349
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,186,197,212,223,243,259,277,304,333,361
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,409,454,511,584,716,847,986,1171,1251,1320
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,267,308,334,370,376,390,428,439,466,501
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,4,5,7,7,7,8,8,8,10,14


#### Remove US from Global Data

In [58]:
confirmed_global_df = confirmed_global_df[confirmed_global_df['Country/Region'] != 'US']
deaths_global_df = deaths_global_df[deaths_global_df['Country/Region'] != "US"]

#### Get list of dates

In [59]:
dates = confirmed_global_df.columns[4:]
dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
       '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20',
       '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20',
       '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20',
       '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20',
       '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20', '4/5/20'],
      dtype='object')

#### Replace NaN Values with Country Region if Province/State or Admin2 is empty

In [60]:
### Remove NaN values from Province/State Column
confirmed_global_df.loc[confirmed_global_df['Province/State'].isnull(),'Province/State'] = confirmed_global_df['Country/Region']
confirmed_states_df.loc[confirmed_states_df['Province_State'].isnull(),'Province_State'] = confirmed_states_df['Country_Region']
confirmed_states_df.loc[confirmed_states_df['Admin2'].isnull(),'Admin2'] = confirmed_states_df['Province_State']
confirmed_states_df.loc[confirmed_states_df['FIPS'].isnull(),'FIPS'] = confirmed_states_df['Province_State']

deaths_global_df.loc[deaths_global_df['Province/State'].isnull(),'Province/State'] = deaths_global_df['Country/Region']
deaths_states_df.loc[deaths_states_df['Province_State'].isnull(),'Province_State'] = deaths_states_df['Country_Region']
deaths_states_df.loc[deaths_states_df['Admin2'].isnull(),'Admin2'] = deaths_states_df['Province_State']

recovered_df.loc[recovered_df['Province/State'].isnull(),'Province/State'] = recovered_df['Country/Region']

#### Remove '.' at end of locations/regions/states

In [61]:

confirmed_global_df['Province/State'] = confirmed_global_df['Province/State'].str.replace('.','')
confirmed_states_df['Province_State'] = confirmed_states_df['Province_State'].str.replace('.','')
confirmed_states_df['Admin2'] = confirmed_states_df['Admin2'].str.replace('.','')

deaths_global_df['Province/State'] = deaths_global_df['Province/State'].str.replace('.','')
deaths_states_df['Province_State'] = deaths_states_df['Province_State'].str.replace('.','')
deaths_states_df['Admin2'] = deaths_states_df['Admin2'].str.replace('.','')

recovered_df['Province/State'] = recovered_df['Province/State'].str.replace('.','')


### Post Object

Holds a list of documents that will be posted to the database

In [62]:
post = {'data': {}}

for x in dates:
    
    post['data'][x] = []
    


### STATES DATA

In [63]:
for x in tqdm(dates):
    for index, row in confirmed_states_df.iterrows():
        date = x
        fips = row['FIPS']
        location = row['Admin2']
        region = row['Country_Region']
        state = row['Province_State']
        lat = row['Lat']
        lng = row['Long_']
        confirmed = row[x]
        if confirmed < 1:
            continue
        else:
            obj = { 
                    'date': pd.to_datetime(x),
                    'formatted_date': x,
                    'location': location,
                    'fips': fips,
                    'state': state,
                    'region': region,
                    'lat': lat,
                    'lng': lng,
                    'confirmed': int(confirmed)
                }
            post['data'][x].append(obj)
    for index, row in deaths_states_df.iterrows():
        date = x
        fips = row['FIPS']
        location = row['Admin2']
        deaths = row[x]
        
        for i in post['data'][x]:
            if i['fips'] == fips and i['location'] == location:
                i['deaths'] = deaths
                    



  0%|                                                                                           | 0/75 [00:00<?, ?it/s]

  1%|█                                                                                  | 1/75 [00:00<00:43,  1.70it/s]

  3%|██▏                                                                                | 2/75 [00:01<00:44,  1.65it/s]

  4%|███▎                                                                               | 3/75 [00:01<00:45,  1.59it/s]

  5%|████▍                                                                              | 4/75 [00:02<00:43,  1.62it/s]

  7%|█████▌                                                                             | 5/75 [00:03<00:42,  1.63it/s]

  8%|██████▋                                                                            | 6/75 [00:03<00:41,  1.65it/s]

  9%|███████▋                                                                           | 7/75 [00:04<00:41,  1.66it/s]

 11%|████████▊                

 89%|█████████████████████████████████████████████████████████████████████████▎        | 67/75 [00:48<00:10,  1.35s/it]

 91%|██████████████████████████████████████████████████████████████████████████▎       | 68/75 [00:49<00:09,  1.41s/it]

 92%|███████████████████████████████████████████████████████████████████████████▍      | 69/75 [00:51<00:09,  1.51s/it]

 93%|████████████████████████████████████████████████████████████████████████████▌     | 70/75 [00:53<00:08,  1.63s/it]

 95%|█████████████████████████████████████████████████████████████████████████████▋    | 71/75 [00:55<00:06,  1.65s/it]

 96%|██████████████████████████████████████████████████████████████████████████████▋   | 72/75 [00:56<00:05,  1.68s/it]

 97%|███████████████████████████████████████████████████████████████████████████████▊  | 73/75 [00:58<00:03,  1.70s/it]

 99%|████████████████████████████████████████████████████████████████████████████████▉ | 74/75 [01:00<00:01,  1.73s/it]

100%|███████████████████████████

### GLOBAL DATA

In [64]:
for x in tqdm(dates):
    for index, row in confirmed_global_df.iterrows():
        date = x
        location = row['Province/State']
        region = row['Country/Region']
        lat = row['Lat']
        lng = row['Long']
        confirmed = row[x]
        if confirmed < 1:
            continue
        else:
            obj = { 
                'date': pd.to_datetime(x),
                'formatted_date': x,
                'location': location,
                'region': region,
                'lat': lat,
                'lng': lng,
                'confirmed': int(confirmed)
            }
            post['data'][x].append(obj)
    
    for index, row in deaths_global_df.iterrows():
        date = x
        location = row['Province/State']
        region = row['Country/Region']
        deaths = row[x]
        
        for i in post['data'][x]:
            if i['location'] == location and i['region'] == region:
                i['deaths'] = deaths
                
    for index, row in recovered_df.iterrows():
        date = x
        location = row['Province/State']
        region = row['Country/Region']
        recovered = row[x]
        
        for i in post['data'][x]:
            if i['location'] == location and i['region'] == region:
                i['recovered'] = recovered               



  0%|                                                                                           | 0/75 [00:00<?, ?it/s]

  3%|██▏                                                                                | 2/75 [00:00<00:05, 12.85it/s]

  5%|████▍                                                                              | 4/75 [00:00<00:05, 12.73it/s]

  8%|██████▋                                                                            | 6/75 [00:00<00:05, 12.55it/s]

 11%|████████▊                                                                          | 8/75 [00:00<00:05, 12.49it/s]

 13%|██████████▉                                                                       | 10/75 [00:00<00:05, 12.44it/s]

 16%|█████████████                                                                     | 12/75 [00:00<00:05, 12.28it/s]

 19%|███████████████▎                                                                  | 14/75 [00:01<00:05, 11.83it/s]

 21%|█████████████████▍       

#### Add 0 to entries without recovered or death data

In [65]:
for x in post['data']:

    for z in post['data'][x]:

        try:
            if z['recovered']:
                pass
        except:
            z['recovered'] = 0
        
        try:
            if z['deaths']:
                pass
        except:
            z['deaths'] = 0

In [66]:
import pymongo
import dns
username = 'Terra925'
password = 'H%40mmond271'

conn = 'mongodb+srv://' + username +':' + password + '@cluster0-paegd.mongodb.net/test?retryWrites=true&w=majority'
client = pymongo.MongoClient(conn)
db = client['corona_virus']
collection = db['daily_cases']

In [67]:
for x in tqdm(post['data']):
    for z in post['data'][x]:
        id = collection.insert_one(z).inserted_id



  0%|                                                                                           | 0/75 [00:00<?, ?it/s]

  1%|█                                                                                  | 1/75 [00:01<02:21,  1.91s/it]

  3%|██▏                                                                                | 2/75 [00:03<02:14,  1.84s/it]

  4%|███▎                                                                               | 3/75 [00:05<02:11,  1.83s/it]

  5%|████▍                                                                              | 4/75 [00:07<02:11,  1.86s/it]

  7%|█████▌                                                                             | 5/75 [00:09<02:17,  1.97s/it]

  8%|██████▋                                                                            | 6/75 [00:11<02:23,  2.08s/it]

  9%|███████▋                                                                           | 7/75 [00:14<02:31,  2.23s/it]

 11%|████████▊                

 89%|█████████████████████████████████████████████████████████████████████████▎        | 67/75 [18:16<10:41, 80.23s/it]

 91%|██████████████████████████████████████████████████████████████████████████▎       | 68/75 [19:52<09:55, 85.02s/it]

 92%|███████████████████████████████████████████████████████████████████████████▍      | 69/75 [21:29<08:52, 88.82s/it]

 93%|████████████████████████████████████████████████████████████████████████████▌     | 70/75 [23:16<07:49, 94.00s/it]

 95%|█████████████████████████████████████████████████████████████████████████████▋    | 71/75 [25:05<06:35, 98.75s/it]

 96%|█████████████████████████████████████████████████████████████████████████████▊   | 72/75 [26:57<05:07, 102.65s/it]

 97%|██████████████████████████████████████████████████████████████████████████████▊  | 73/75 [28:52<03:32, 106.44s/it]

 99%|███████████████████████████████████████████████████████████████████████████████▉ | 74/75 [30:50<01:49, 109.74s/it]

100%|███████████████████████████